# 참고자료

http://jorditorres.org/first-contact-with-tnesorflow/#cap4 (First Contact with tensorflow)

https://deepstat.tistory.com/9 (단일신경망 Single Layer Neural Network (ver. Tensorflow for Python)

https://deepstat.tistory.com/10 (단일신경망 Single Layer Neural Network (ver. Tensorflow for R)

http://fluxml.ai/ (flux: The Elegant Machine Learning Stack)

# 단일신경망 Single Layer Neural Network (ver. Flux in Julia)

## The MNIST data-set

In [1]:
using PyCall
@pyimport tensorflow.keras.datasets.mnist as MNIST

In [2]:
mnist_train, mnist_test = MNIST.load_data()
println(typeof(mnist_train))
println(typeof(mnist_test))

Tuple{Array{UInt8,3},Array{UInt8,1}}
Tuple{Array{UInt8,3},Array{UInt8,1}}


In [3]:
mnist_train[1][1,:,:]

28×28 Array{UInt8,2}:
 0x00  0x00  0x00  0x00  0x00  0x00  …  0x00  0x00  0x00  0x00  0x00  0x00
 0x00  0x00  0x00  0x00  0x00  0x00     0x00  0x00  0x00  0x00  0x00  0x00
 0x00  0x00  0x00  0x00  0x00  0x00     0x00  0x00  0x00  0x00  0x00  0x00
 0x00  0x00  0x00  0x00  0x00  0x00     0x00  0x00  0x00  0x00  0x00  0x00
 0x00  0x00  0x00  0x00  0x00  0x00     0x00  0x00  0x00  0x00  0x00  0x00
 0x00  0x00  0x00  0x00  0x00  0x00  …  0xf7  0x7f  0x00  0x00  0x00  0x00
 0x00  0x00  0x00  0x00  0x00  0x00     0xc3  0x40  0x00  0x00  0x00  0x00
 0x00  0x00  0x00  0x00  0x00  0x00     0x27  0x00  0x00  0x00  0x00  0x00
 0x00  0x00  0x00  0x00  0x00  0x00     0x00  0x00  0x00  0x00  0x00  0x00
 0x00  0x00  0x00  0x00  0x00  0x00     0x00  0x00  0x00  0x00  0x00  0x00
 0x00  0x00  0x00  0x00  0x00  0x00  …  0x00  0x00  0x00  0x00  0x00  0x00
 0x00  0x00  0x00  0x00  0x00  0x00     0x00  0x00  0x00  0x00  0x00  0x00
 0x00  0x00  0x00  0x00  0x00  0x00     0x00  0x00  0x00  0x00  0x00  0x00
   

In [4]:
mnist_test[1][1]

0x00

### `next_batch`라는 함수 만들기.

In [5]:
mutable struct My_data
    data::Tuple
    start_n::Int
end

In [6]:
function next_batch(data::My_data, n::Int)
    start_n = data.start_n
    end_n = data.start_n + n - 1
    batch_X = data.data[1][start_n:end_n,:,:]
    batch_Y = data.data[2][start_n:end_n]
    data.start_n = (end_n+1) % (size(data.data[1])[1])
    return float(reshape(batch_X,:,784)), float(batch_Y)
end

next_batch (generic function with 1 method)

In [7]:
train_dat = My_data(mnist_train,1)
test_dat = My_data(mnist_test,1);

## Single layer neural network

In [8]:
using Flux

#### 변수 지정

In [9]:
W = param(zeros(784,10))
b = param(zeros(1,10));

#### 모형 설정 Activation function : softmax

In [10]:
function model(X)
    yhat = transpose(softmax(transpose(X * W .+ b)))
    return yhat
end

model (generic function with 1 method)

#### Loss function : cross-entropy

In [11]:
function cross_entropy(X,Y)
    yhat = model(X)
    loss = sum(-Y .* log.(yhat))
    return loss
end

cross_entropy (generic function with 1 method)

#### Optimizer : SGD (learning_rate = 0.001)

In [12]:
PARS = params(W,b)

for i = 0:100
    Flux.train!(cross_entropy, [next_batch(train_dat,1000)], SGD((PARS), 0.001))
    if i % 10 == 0
        test_X, test_Y = next_batch(test_dat,10000)
        tmp = findmax(model(test_X).data, dims=2)
        test_acc = sum([tmp[2][j][2] for j in 1:10000] .== test_Y)/10000
        println("step:",i,"  test_acc:" ,test_acc)
    end
end

step:0  test_acc:0.1135
step:10  test_acc:0.1135
step:20  test_acc:0.1135
step:30  test_acc:0.1135
step:40  test_acc:0.1135
step:50  test_acc:0.1135
step:60  test_acc:0.1135
step:70  test_acc:0.1135
step:80  test_acc:0.1135
step:90  test_acc:0.1135
step:100  test_acc:0.1135
